In [1]:
#from https://towardsdatascience.com/the-lesser-known-rising-application-of-llms-775834116477

!git clone https://github.com/shawngraham/structured-cultural-heritage-crime

Cloning into 'structured-cultural-heritage-crime'...
remote: Enumerating objects: 222, done.
remote: Counting objects: 100% (222/222), done.
remote: Compressing objects: 100% (165/165), done.
remote: Total 222 (delta 110), reused 127 (delta 48), pack-reused 0
Receiving objects: 100% (222/222), 1.23 MiB | 8.12 MiB/s, done.
Resolving deltas: 100% (110/110), done.


In [2]:
%cd structured-cultural-heritage-crime

/content/structured-cultural-heritage-crime


In [ ]:
!pip install -r requirements.txt
#should remove mistral and replace with openai


In [ ]:
!pip install langchain-openai


In [5]:
import os
import json
import time
from pathlib import Path

import pandas as pd
from langchain.output_parsers import PydanticOutputParser
#from langchain_mistralai.chat_models import ChatMistralAI
from langchain_openai import ChatOpenAI
#from dotenv import load_dotenv

from core import run
from prompt import DEFAULT_BASE_PROMPT, create_prompt
from schemas import CulturalHeritageSchema


In [6]:

from google.colab import userdata
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

## Upload Texts

The texts should have coreference resolution done as much as possible before processing them here. Use the other notebook?

In [ ]:
!unzip coref-tc.zip

In [ ]:
import os
import pandas as pd

def read_files_to_dataframe(folder_path: str) -> pd.DataFrame:
    # List all text files in the folder
    file_names = [f for f in os.listdir(folder_path) if f.endswith('.txt')]

    # Read each file's content into a list
    data = []
    for file_name in file_names:
        with open(os.path.join(folder_path, file_name), 'r', encoding='utf-8') as file:
            content = file.read()
            data.append(content)

    # Create a DataFrame with the file contents
    df = pd.DataFrame(data, columns=['content'])

    return df

# Usage example
folder_path = 'resolved'
df = read_files_to_dataframe(folder_path)
print(df)

                                               content
0    Two stolen alabaster duck vessels returned to ...
1    A Fremont Culture figurine stolen from the col...
2    Also known as : Huaca Rajada \n\n  One of the ...
3    Also known as : Getty Museum Returns to Italy ...
4    Discovery of Viking - age silver , looted from...
..                                                 ...
135  One of the most spectacular hoard discoveries ...
136  Also known as : Momia de Cerro El Plomo \n\n  ...
137  In January 2008 , US federal agents raided the...
138  Classic Maya stone sculpture from the site of ...
139  It was said that Larisa Zavadskaya had been re...

[140 rows x 1 columns]


In [10]:
## may 27 - let's try a csv
import pandas as pd
df = pd.read_csv('/content/openartdata-latchford-et-al.csv')

In [11]:
df

,Selection,Source,RetrievalDate,YEAR Acquired,Source Url,Title,Date,Artist,Credit Line,Acc Num,Provenance,Description,Artist_Region,Artwork__tombstone Value,Provenance Plus
0,Brummer,Metmuseum,29March2022,2020.0,https://www.metmuseum.org/art/collection/searc...,Warrior Figure | Wari | The Metropolitan Museu...,A.D. 500–800,Wari,"Gift of the Mol Collection, 2020",2020.386.2,"G. R. Schmidt, Lima, until 1929, [The Brummer ...",This small turquoise figurine represents a sta...,NaN,NaN,NaN
1,Klejman,Metmuseum,29March2022,2011.0,https://www.metmuseum.org/art/collection/searc...,Face Mask | Yup'ik | The Metropolitan Museum o...,late 19th century,Yup'ik,"Ralph T. Coe Collection, Gift of Ralph T. Coe ...",2011.154.158,Collected in late 19th century by Bishop Farho...,NaN,NaN,NaN,Collected in late 19th century by Bishop Farho...
2,Klejman,Metmuseum,29March2022,2011.0,https://www.metmuseum.org/art/collection/searc...,Face Mask | Yup'ik | The Metropolitan Museum o...,late 19th century,Yup'ik,"Ralph T. Coe Collection, Gift of Ralph T. Coe ...",2011.154.157,Collected in late 19th century by Bishop Farho...,NaN,NaN,NaN,Collected in late 19th century by Bishop Farho...
3,Klejman,Metmuseum,29March2022,2011.0,https://www.metmuseum.org/art/collection/searc...,Bannerstone | Archaic | The Metropolitan Museu...,5000–4000 B.C.,Archaic,"Ralph T. Coe Collection, Gift of Ralph T. Coe ...",2011.154.13,"Collected in Kankakee County, Illinois (?), Ph...",NaN,NaN,NaN,"Collected in Kankakee County, Illinois (?), Ph..."
4,Klejman,Metmuseum,29March2022,2011.0,https://www.metmuseum.org/art/collection/searc...,Marble statuette of young Dionysos | Roman | I...,1st–2nd century A.D.,Roman,"Gift of Edward Klagsbrun and Joan Saslow, in m...",2011.517,"[Until 1967, with John J. Klejman, New York], ...",Adaptation of a Greek statue type of the late ...,NaN,NaN,"[Until 1967, with John J. Klejman, New York], ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
860,Brummer,Metmuseum,29March2022,NaN,https://www.metmuseum.org/art/collection/searc...,Mask for Men's Association (probably Bo nun am...,19th - early 20th century (before 1912),Baule or Lagoon peoples,"The University of Pennsylvania Museum, Philade...",ON LOAN TO THE MET,"Joseph Brummer, Paris, before 1912, Charles Vi...",This zoomorphic mask entered the holdings of t...,NaN,NaN,NaN
861,NaN,Metmuseum,29March2022,NaN,https://www.metmuseum.org/art/collection/searc...,"Georges Braque | Violin and Sheet Music: ""Peti...","Paris, early 1913",Georges Braque,"The Metropolitan Museum of Art, New York, Prom...",CTO.127,"[Galerie Kahnweiler, Paris, before June 1913–1...",Braque depicted violins more than any other in...,French,"Violin and Sheet Music: ""Petit Oiseau"";Artist;...",NaN
862,Klejman,Metmuseum,29March2022,NaN,https://www.metmuseum.org/art/collection/searc...,Seated Female Figure | Bamana peoples | The Me...,19th–mid-20th century (?),Bamana peoples,"The Menil Collection, Houston; RETURNED TO LENDER",SL.3.2020.11.2,"J. J. Klejman, New York, John and Dominique de...","In Bamana society, individuals can acquire mys...",NaN,NaN,"J. J. Klejman, New York, John and Dominique de..."
863,Klejman,Metmuseum,29March2022,NaN,https://www.metmuseum.org/art/collection/searc...,Mother and Child Figure | Dogon peoples | The ...,17th–19th century (?),Dogon peoples,"The Menil Collection, Houston; RETURNED TO LENDER",SL.3.2020.11.4,"[J. J. Klejman, New York], John and Dominique ...","As the focus of prayers and offerings, Dogon s...",NaN,NaN,"[J. J. Klejman, New York], John and Dominique ..."


In [13]:
import importlib
import sys
importlib.reload(sys.modules['prompt'])
importlib.reload(sys.modules['schemas'])
from prompt import DEFAULT_BASE_PROMPT, create_prompt
from schemas import CulturalHeritageSchema

model_name = "gpt-4o"
temperature = 0
llm = ChatOpenAI(api_key=OPENAI_API_KEY, model_name=model_name, temperature=temperature)
parser = PydanticOutputParser(pydantic_object=CulturalHeritageSchema)
#prompt = create_prompt(DEFAULT_BASE_PROMPT, parser, df["content"][0]) #for when the df is the individual text files loaded in
prompt = create_prompt(DEFAULT_BASE_PROMPT, parser, df.iloc[0]) #trying to read in the first row of a regular csv, for testin
prompt


[HumanMessage(content='\nExtract information regarding cultural heritage crime from the following text.\n\nIn particular, please provide the following information:\n- A list of triplets, where each triplet consists of two entities and the relation between them. Only use the predefined relations listed below and provide detailed descriptions for each entity.\n\nThe predefined relations are:\n- \'is_the_owner_of\': Denotes a business relationship where an actor controls, or is the legal owner of, a business, gallery, auction house, or other for-profit organization.\n- \'works_with\': Denotes a business relationship between actors who are dealers, organizations, looters, or collectors.\n- \'works_for\': Describes an employment or contractual relationship between actors who are dealers, organizations, businesses, museums, government agencies, looters, or collectors.\n- \'has_possession_of\': Describes a situation where a dealer, organization, collector, or auction house controls an artefac

In [14]:
example = await run(llm, prompt, parser)
example

CulturalHeritageSchema(triplets=[Triplet(entity1=Entity(name='The Metropolitan Museum of Art'), relation=Relation(name=<RelationType.has_possession_of: 'has_possession_of'>), entity2=Entity(name='Warrior Figure')), Triplet(entity1=Entity(name='The Mol Collection'), relation=Relation(name=<RelationType.donates_to: 'donates_to'>), entity2=Entity(name='The Metropolitan Museum of Art')), Triplet(entity1=Entity(name='Warrior Figure'), relation=Relation(name=<RelationType.comes_from: 'comes_from'>), entity2=Entity(name='Wari')), Triplet(entity1=Entity(name='G. R. Schmidt'), relation=Relation(name=<RelationType.has_possession_of: 'has_possession_of'>), entity2=Entity(name='Warrior Figure')), Triplet(entity1=Entity(name='The Brummer Gallery'), relation=Relation(name=<RelationType.has_possession_of: 'has_possession_of'>), entity2=Entity(name='Warrior Figure'))])

In [17]:
# ## on all the csv?
raw_outputs = []
outputs = []

for i in range(df.shape[0]):

     prompt = create_prompt(DEFAULT_BASE_PROMPT, parser, df.iloc[i])
     result = await run(llm, prompt, parser)
     raw_outputs.append(result)

     output_map = {
         "article_id":i,
     }

     if isinstance(result, CulturalHeritageSchema):
         output =  output_map | result.dict()
         outputs.append(output)
     else:
         output_map["raw_llm_output"] = result
         outputs.append(output_map)

     time.sleep(1)

Error in parsing Failed to parse CulturalHeritageSchema from completion {"triplets": [{"entity1": {"name": "Seated Female Figure"}, "relation": {"name": "comes_from"}, "entity2": {"name": "Bamana peoples"}}, {"entity1": {"name": "Seated Female Figure"}, "relation": {"name": "has_possession_of"}, "entity2": {"name": "The Menil Collection, Houston"}}, {"entity1": {"name": "The Menil Collection, Houston"}, "relation": {"name": "returned_to"}, "entity2": {"name": "Lender"}}, {"entity1": {"name": "Seated Female Figure"}, "relation": {"name": "has_possession_of"}, "entity2": {"name": "J. J. Klejman, New York"}}, {"entity1": {"name": "Seated Female Figure"}, "relation": {"name": "has_possession_of"}, "entity2": {"name": "John and Dominique de Menil"}}]}. Got: 1 validation error for CulturalHeritageSchema
triplets.2.relation.name
  Input should be 'is_the_owner_of', 'works_with', 'works_for', 'has_possession_of', 'purchases', 'buys_from', 'sells', 'makes_sale_to', 'donates_to', 'obtains_from',

In [22]:
#cost was 8.90 on may 27th with gpt-o
# only two lines with errors
outputs

with open('openartdata-latchford-et-al-out.json', 'w') as outfile:
  outfile.write('\n'.join(str(i) for i in outputs))


In [ ]:
##-----
## for when the df is created by appending individual texts
## iterate over everything in 'texts' folder and write it to a dataframe
import pandas as pd
import asyncio
import nest_asyncio

# Allow nested use of asyncio in environments like Jupyter
nest_asyncio.apply()

raw_outputs = []
outputs = []

# Assuming you have an asynchronous function to run the model
async def process_content(content: str, idx: int, llm, parser):
    prompt = create_prompt(DEFAULT_BASE_PROMPT, parser, content)
    result = await run(llm, prompt, parser)
    raw_outputs.append(result)  # Store raw output for logging or debugging purposes

    output_map = {
        "article_id": idx,
    }

    if isinstance(result, CulturalHeritageSchema):
        output = output_map | result.dict()
        outputs.append(output)
    else:
        output_map["raw_llm_output"] = result
        outputs.append(output_map)

    return result

async def main():
    # Placeholder for the processed triplets
    processed_triplets = []

    for i in range(df.shape[0]):
        content = df["content"][i]
        result = await process_content(content, i, llm, parser)

        # Only continue if the result is of the expected type
        if isinstance(result, CulturalHeritageSchema):
            for triplet in result.triplets:
                processed_triplets.append({
                    "article_id": i,
                    "entity1": triplet.entity1.name,
                    "relation": triplet.relation.name,
                    "entity2": triplet.entity2.name
                })

    # Create DataFrame with columns entity1, relation, entity2
    result_df = pd.DataFrame(processed_triplets, columns=["article_id", "entity1", "relation", "entity2"])

    return result_df

# Function to run the async function in both environments
def run_main_loop():
    # Get or create the event loop
    try:
        # If there's already an event loop, use it
        loop = asyncio.get_running_loop()
    except RuntimeError:
        # If no event loop is running, create one
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)

    return loop.run_until_complete(main())

# Run the main async function
result_df = run_main_loop()
print(result_df)

Error in parsing Failed to parse CulturalHeritageSchema from completion {"triplets": [{"entity1": {"name": "J. Paul Getty Museum"}, "relation": {"name": "has_possession_of"}, "entity2": {"name": "Fifth-century BC Attic red-figured kylix"}}, {"entity1": {"name": "J. Paul Getty Museum"}, "relation": {"name": "has_possession_of"}, "entity2": {"name": "Torso of a second-century AD statue of the god Mithra"}}, {"entity1": {"name": "J. Paul Getty Museum"}, "relation": {"name": "has_possession_of"}, "entity2": {"name": "Second-century AD copy of a head of Diadoumenos by Polykleitos"}}, {"entity1": {"name": "J. Paul Getty Museum"}, "relation": {"name": "purchases"}, "entity2": {"name": "Fifth-century BC Attic red-figured kylix"}}, {"entity1": {"name": "J. Paul Getty Museum"}, "relation": {"name": "purchases"}, "entity2": {"name": "Torso of a second-century AD statue of the god Mithra"}}, {"entity1": {"name": "J. Paul Getty Museum"}, "relation": {"name": "purchases"}, "entity2": {"name": "Secon

Cost: $3.12 for files in coref-tc.zip

In [ ]:
result_df.to_csv('out.csv', index=True, sep=";")